In [ ]:
!pip install lark-parser -q

In [ ]:
# This script is inspired by Paola's script
# It implements an Earley algorithm parser for a simple "Minion language" grammar

import lark

# Define a context-free grammar using Lark's EBNF syntax
# This grammar describes a simple "Minion language" with specific rules
grammar_string = """
    ?start: sentence            # Define the starting symbol for the grammar

    sentence: phrase            # A sentence consists of a phrase

    phrase: term phrase_prime   # A phrase is a term followed by an optional phrase_prime
    ?phrase_prime: "po" term phrase_prime | // Epsilon is implicit (empty alternative)

    term: action                # A term is an action
    action: greeting | farewell | request  # An action can be a greeting, farewell, or request
    request: "me" "want" noun   # A request is "me want" followed by a noun

    greeting: "bello"           # A greeting is the word "bello"
    farewell: "papaya"          # A farewell is the word "papaya"
    noun: "banana" | "bapple"   # A noun is either "banana" or "bapple"

    %ignore " "  // Ignore spaces between tokens (whitespace handling)
"""

# Create a Lark parser using the grammar defined above
# The parser uses the Earley algorithm, which is Lark's default
# 'start' parameter defines which rule to use as the starting point
parser = lark.Lark(grammar_string, start='sentence')

original_sentences = [
    # These sentences should be successfully parsed according to our grammar
    "Bello",                           # Simple greeting
    "papaya",                          # Simple farewell
    "Me want banana",                  # Simple request for banana
    "Me want bapple",                  # Simple request for bapple
    "Bello po Me want banana",         # Greeting followed by request
    "Me want banana po papaya",        # Request followed by farewell
    "Bello po Me want banana po papaya", # Greeting, request, and farewell combined

    # These sentences should fail to parse according to our grammar
    "banana want Me",                  # Incorrect word order
    "po Bello",                        # 'po' cannot start a phrase
    "Me want",                         # Incomplete request (missing noun)
    "Bello po papaya po",              # Trailing 'po' is invalid
    "banana",                          # Single noun is not a valid sentence
    "Me want banana Po Me want bapple", # Case sensitivity issue with 'Po' vs 'po'
    ""                                 # Empty string is invalid
]

# If the parser was successfully created, process all test sentences
if parser:
    for sentence in original_sentences:
        print(" ")  
        print("Input Sentence:", sentence)  

        # Convert the sentence to lowercase for case-insensitive parsing
        sentence_lowercase = sentence.lower()

        try:
            tree = parser.parse(sentence_lowercase)
            print("Parse Tree:")
            print(tree.pretty())  

        except lark.exceptions.UnexpectedInput as e:
            print(f"Unable to parse: {e}")



 
Input Sentence: Bello
Parse Tree:
sentence
  phrase
    term
      action
        greeting
    phrase_prime

 
Input Sentence: papaya
Parse Tree:
sentence
  phrase
    term
      action
        farewell
    phrase_prime

 
Input Sentence: Me want banana
Parse Tree:
sentence
  phrase
    term
      action
        request
          noun
    phrase_prime

 
Input Sentence: Me want bapple
Parse Tree:
sentence
  phrase
    term
      action
        request
          noun
    phrase_prime

 
Input Sentence: Bello po Me want banana
Parse Tree:
sentence
  phrase
    term
      action
        greeting
    phrase_prime
      term
        action
          request
            noun
      phrase_prime

 
Input Sentence: Me want banana po papaya
Parse Tree:
sentence
  phrase
    term
      action
        request
          noun
    phrase_prime
      term
        action
          farewell
      phrase_prime

 
Input Sentence: Bello po Me want banana po papaya
Parse Tree:
sentence
  phrase
    term
 